## Lab 4 - part 2: Data Weights and Autocorellation

**TU Delft and WUR**<br>
**Q1 2024**<br>
**Instructor:** Theodoros Chatzivasileiadis <br>
**Instructor:** Hans Hoogenboom <br>
**TA:** Ka Yi Chua <br>
**[Metropolitan Data 1](https://jhoogenboom.github.io/spatial-data-science/_index.html)** <br>



## Exercise 4a: Spatial Autocorrelation and ESDA 

USE the Liverpool IMD data we have been using in previous sessions. This will require you to:

* Load up the IMD dataset for Liverpool.
* Create a choropleth of the `imd_score` variable.
* Compute the spatial weights matrix for the LSOAs. Think of one criterium to build it that you think would fit this variable (e.g. contiguity, distance-based, etc.), and apply it.
* Create the standardized version of the IMD scores.
* Calculate the spatial lag of the standardized scores.
* Create the Moran Plot.
* Calculate the value of Moran's I as well as its significance level.
* Perform a LISA analysis and generate a map of the results. What are the main patterns?

## BASICS

Make sure we have all the tools we need

In [1]:
# make sure we have all the plug ins I need for the exercises.

%matplotlib inline
import seaborn as sns
import pandas as pd
import pysal as ps
from pysal.explore import esda
from pysal.lib import weights
from libpysal.io import open as psopen
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation
import contextily as ctx

c:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\spaghetti\network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)
c:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Question 1: Build a Contiguity-Based Weight Matrix

I modified my code to be more similar to Marnix's. I had done all of these steps, but I hadn't defined a function.

In [2]:
# QUESTION 1: reads in shapefile of polygons from LIVERPOOL

Liverpool_imd = 'C:\\Users\\natha\\Downloads\\lab-04\\data\\IMD\\IMD\\lab04_imd.shp'
lookup_path = 'C:\\Users\\natha\\Downloads\\lab-04\\data\\IMD\\IMD\\OAs'
Liverpool = gpd.read_file(Liverpool_imd).set_index('lsoa11cd', drop=False)  # so we can access the data quickly later

def Contiguity_Matrix(path):                                                        # define a function that creates a contiguity based weight matrix with a file path
    Liverpool = gpd.read_file(Liverpool_imd).set_index('lsoa11cd', drop=False)      # read Liverpool data and set the index
    w_queen = weights.Queen.from_dataframe(Liverpool, idVariable='lsoa11cd')        # construct a contiguity based weight matrix
    return w_queen                                                                  # return the functions

w_queen = Contiguity_Matrix(Liverpool_imd)

C:\Users\natha\AppData\Local\Temp\ipykernel_9736\1117236640.py:10: FutureWarning: `idVariable` is deprecated and will be removed in future. Use `ids` instead.
  w_queen = weights.Queen.from_dataframe(Liverpool, idVariable='lsoa11cd')        # construct a contiguity based weight matrix
c:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There is 1 island with id: E01019077.
  W.__init__(self, neighbors, ids=ids, **kw)


In [3]:
# QUESTION 1: Return the weight matrix
w_queen['E01006690']

{'E01006720': 1.0,
 'E01006691': 1.0,
 'E01006695': 1.0,
 'E01006697': 1.0,
 'E01033763': 1.0,
 'E01006692': 1.0,
 'E01006759': 1.0}

## Drop Islands to facilitate future calculations

In [4]:
# w_queen.islands       # I ran this to confirm if there is an island in the data

Liverpool_clean = Liverpool.drop(w_queen.islands)

In [5]:
# Must re-calculate the weight matrix now that the island has been removed.
w_queen = weights.Queen.from_dataframe(Liverpool_clean, idVariable='lsoa11cd')
w_queen

C:\Users\natha\AppData\Local\Temp\ipykernel_9736\2348843082.py:2: FutureWarning: `idVariable` is deprecated and will be removed in future. Use `ids` instead.
  w_queen = weights.Queen.from_dataframe(Liverpool_clean, idVariable='lsoa11cd')
c:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 7 disconnected components.
  W.__init__(self, neighbors, ids=ids, **kw)


## Calculate Moran's I

Asal's code - but drop the extra stuff. 

We decided NOT to Print Moran's I is: and Moran's P is: because if we want to do a calulation later with the result of the function, we won't be able to do future calculations.

In [29]:
# Question 2: Calculate Moran's I

# mi = esda.Moran(Liverpool_clean['IMD_Rank'], w_queen)
# mi.I

def Calculate_MoranI(W, y):
    mi = esda.Moran(y, W)
    morans_i_value = mi.I
    morans_p_value = mi.p_sim
    return morans_i_value, morans_p_value

p = Calculate_MoranI(w_queen, Liverpool_clean['IMD_Rank'])

print("Moran's I is:", p[0])
print("P Value is:", p[1])

Moran's I is: 0.5895602570187034
P Value is: 0.001


## Question 3: Standardizing Spatial Matrices

Modifed mine to be like Asal's and Marnix's because there's is in a function

In [30]:
# QUESTION 3: Standardizing Spatial Matrices

# w_queen.transform = 'R'

def Standardize_Spatial_Matrix(w):
    """
    create a function that takes a weight matrix and creates a standardized spatal matrix.
    """
    w.transform = 'R'
    return w

Standardize_Spatial_Matrix(w_queen)

In [31]:
# QUESTION 3: test to make sure the function works
w_queen['E01006690']

{'E01006720': 0.14285714285714285,
 'E01006691': 0.14285714285714285,
 'E01006695': 0.14285714285714285,
 'E01006697': 0.14285714285714285,
 'E01033763': 0.14285714285714285,
 'E01006692': 0.14285714285714285,
 'E01006759': 0.14285714285714285}

## Question 4: Distance Based Weights

Take my response and make it a function, like Asal and Marnix

In [36]:
# QUESTION 4: Distance Based Weights

# w_10km_dist = weights.DistanceBand.from_dataframe(Liverpool, 1000, binary=False)

def create_distance_weights(Liverpool, threshold_distance):
    distance_weights = weights.DistanceBand.from_dataframe(Liverpool, threshold_distance, binary=False)
    return distance_weights

w_1km_dist = create_distance_weights(Liverpool, 1000)

In [37]:
# Question 4: Test
w_1km_dist['E01006690'] # long output - many neighbours

{'E01006691': 0.0013256933794006658,
 'E01006692': 0.0016882958052879748,
 'E01006695': 0.0011204624346602458,
 'E01006697': 0.001402472386182104,
 'E01006720': 0.0013393372744865168,
 'E01006725': 0.0010103380475945504,
 'E01006726': 0.001056402257271986,
 'E01033763': 0.0012954566504819575}

## Question 5: Visualize a Moran Plot



In [40]:
def plot_moran(W, attribute):
    # Setup the figure and axis
    W = standardize_weights(W)
    attribute = np.array(attribute)
    attribute_st = (attribute - attribute.mean()) / attribute.std()
    w_queen_score = weights.lag_spatial(W, attribute_st)
    # Plot values
    sns.regplot(x=attribute_st, y=w_queen_score, data=Liverpool, ci=None)
    plt.show()
plot_moran(w_queen, attribute)



NameError: name 'attribute' is not defined

In [45]:
# QUESTION 5: Visualize a Moran plot

def plot_moran(W, y, output_file='moran_plot.png'):

  # Standardize the IMD scores  
    st_Liverpool = (y - y.mean()) / y.std()  
    
    # Compute the spatial lag of the standardized version  
    st_w_Liverpool = Standardize_Spatial_Matrix(W)
    st_w_score = weights.lag_spatial(st_w_Liverpool, st_Liverpool)
    # Setup the figure and axis for the Moran plot  
    f, ax = plt.subplots(1)  
    
    # Plot values for the Moran plot  
    sns.regplot(x=st_Liverpool, y=st_w_score, ax=ax, ci=None)  
    
    # Add vertical and horizontal lines  
    plt.axvline(0, c='k', alpha=0.5)  
    plt.axhline(0, c='k', alpha=0.5)  

    # Set labels and title  
    ax.set_xlabel("Standardized IMD Score")  
    ax.set_ylabel("Spatial Lag of Standardized IMD Score")  
    plt.title("Moran's I Plot")  

    # Display the Moran plot  
    plt.show()  
    
    # Save the plot as an image file  
    plt.savefig(output_file)  
    plt.close()
    

# imd is your DataFrame containing 'IMDScore'  
# w_queen is your spatial weights matrix (e.g., from create_distance_weights)  
plot_moran(w_queen, Liverpool['IMDScore'], output_file='moran_plot.png') 

ValueError: dimension mismatch

In [43]:

# imd is your DataFrame containing 'IMDScore'  
# w_queen is your spatial weights matrix (e.g., from create_distance_weights)  
plot_moran(Liverpool, w_queen, output_file='moran_plot.png') 
 

# Row-standardize the queen matrix
w_queen.transform = 'R'
# Compute spatial lag of `imd_score`
w_queen_score = weights.lag_spatial(w_queen, Liverpool['IMDScore'])
# Print the first five elements
w_queen_score[:5]

Liverpool['w_queen_score'] = w_queen_score

# Standardize the IMD scores
std_imd = (Liverpool['IMDScore'] - Liverpool['IMDScore'].mean()) / Liverpool['IMDScore'].std()
# Compute the spatial lag of the standardized version and save is as a 
# Series indexed as the original variable
std_w_imd = pd.Series(weights.lag_spatial(w_queen, std_imd), index=std_imd.index)
# Setup the figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot values
sns.regplot(x=std_imd, y=std_w_imd, ci=None)
# Add vertical and horizontal lines
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
# Display
plt.show()

AttributeError: 'Queen' object has no attribute 'mean'

For this part I would like you to experiment with Data from Amstedam. However this will require you to find the dataset yourself!

## Question 1: Building a Contiguity-Based Weight Matrix

Use PySAL to create a contiguity-based weight matrix (W) for a given spatial dataset of polygons. Write a function create_contiguity_weights that:

    Reads in a shapefile of polygons.
    Constructs a Queen contiguity-based spatial weights matrix.
    Returns the weight matrix.

## Question 2: Calculating Moran’s I

Write a function calculate_morans_i that:

    Takes a spatial weight matrix W and an attribute array y.
    Calculates Moran’s I for the given attribute array.
    Returns the Moran’s I value and its p-value.

## Question 3: Standardizing Spatial Weight Matrices

Create a function standardize_weights that:

    Takes a spatial weight matrix W.
    Standardizes it so that the weights of each row sum to one.
    Returns the standardized weight matrix.

## Question 4: Creating Distance-Based Weights

Write a function create_distance_weights that:

    Takes a set of point coordinates and a threshold distance.
    Constructs a distance-based spatial weight matrix where all points within the threshold distance are assigned a weight.
    Returns the weight matrix.

## Question 5: Visualizing a Moran Plot

Using PySAL, write a function plot_moran that:

    Takes a spatial weights matrix W and an attribute array y.
    Creates and displays a Moran plot for the attribute.
    Saves the Moran plot as an image file.

In [13]:
# your code here
